In [2]:
from bs4 import BeautifulSoup
from datetime import time
from datetime import datetime
from datetime import timedelta
from collections import Counter
import json
import requests
import csv
import pandas as pd
import numpy as np
import aiohttp
import asyncio
import concurrent.futures
import io
import re

In [3]:
Crossfit_Men = pd.read_csv('/Users/cdmoseley/dai_files/Capstone/Galvanize_Capstone_Crossfit/Data/Analysis_Data/Crossfit_Men.csv')
Crossfit_Men.info()

/var/folders/xm/kc32pwjj5fj5xqf43yb4dz340000gn/T/ipykernel_14964/781522481.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  Crossfit_Men = pd.read_csv('/Users/cdmoseley/dai_files/Capstone/Galvanize_Capstone_Crossfit/Data/Analysis_Data/Crossfit_Men.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176260 entries, 0 to 176259
Data columns (total 46 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Athlete ID               176259 non-null  object 
 1   Gender                   176255 non-null  object 
 2   Competitor Name          176254 non-null  object 
 3   Country Name             175672 non-null  object 
 4   Region Name              176255 non-null  object 
 5   Affiliate Name           155236 non-null  object 
 6   Height (in)              89684 non-null   float64
 7   Age (Years)              176255 non-null  float64
 8   Weight (lbs)             89451 non-null   float64
 9   Overall Rank             176255 non-null  float64
 10  24.1 Rank                176255 non-null  float64
 11  24.2 Rank                176255 non-null  float64
 12  24.3 Rank                176255 non-null  float64
 13  2024 Rank                176257 non-null  float64
 14  2023

In [13]:
Crossfit_Men['Snatch (lbs)'].max()

425.0